In [29]:
import pandas as pd
import numpy as np
import os.path as ospath

In [30]:
excelfile = ospath.join('..','Serious games','Games','БД4_Лёгкая.xlsm')

In [3]:
Alldata = pd.read_excel(excelfile,sheet_name='Данные')
Games = pd.read_excel(excelfile,sheet_name='Игры')
print(Alldata.columns)
Games.columns

Index(['Game', 'Time', 'Subject', 'Group', 'GrSubject', 'Type', 'Gain',
       'Penalty', 'x', 's1', 's2', 's3', 'AcceptAdvice', 'CalcCount',
       'CalcAdviceCount', 'UseHelp'],
      dtype='object')


Index(['Session', 'Game', 'Date', 'Place', 'GamersCount', 'R',
       'Parametr1_alfa', 'Parametr2_beta', 'Mechname', 'GameMech',
       'Cooperation', 'talks', 'Name', 'Datafile', 'Sbjfile', 'стимул',
       'Group', 'Курс балла'],
      dtype='object')

In [4]:
YHGames=Games[(Games.Mechname=='yh') & (Games.GameMech=='agree')]
YHExpGames=Games[(Games.Mechname=='yh') & (Games.GameMech=='exp')]

In [5]:
YHData=Alldata[Alldata.Game.isin(YHGames.Game)]
YHEData=Alldata[Alldata.Game.isin(YHExpGames.Game)]

In [6]:
print(YHData.shape,YHEData.shape)

(1035, 16) (75, 16)


In [85]:
GLGames=Games[(Games.Mechname=='gl') & (Games.GameMech=='agree')]
GLData=Alldata[Alldata.Game.isin(GLGames.Game)]

In [5]:
GLEGames=Games[(Games.Mechname=='gl') & (Games.GameMech=='exp')]
GLEData=Alldata[Alldata.Game.isin(GLEGames.Game)]

In [86]:
GLData.shape

(2847, 16)

## Нэш-торги

In [7]:
YHg=YHData.groupby(['Game','Time'])

In [32]:
YHg.sum()

Group  GrSubject  Type       Gain   Penalty      x   s1     s2  \
Game Time                                                                   
40   1         3          6  35.0   6.812121  11.59900  115.0  0.0  215.0   
     2         3          6  35.0   6.808913  11.62040  115.0  0.0  215.1   
     3         3          6  35.0   6.812121  11.59900  115.0  0.0  215.0   
     4         3          6  35.0  15.951731   2.57165  115.0  0.0  102.6   
     5         3          6  35.0   6.833104  14.15300  115.0  0.0  216.5   
     6         3          6  35.0  11.057325   7.45100  115.0  0.0  172.0   
     7         3          6  35.0  15.000859   3.33200  115.0  0.0  121.1   
     8         3          6  35.0  15.015961   3.34400  115.0  0.0  121.2   
     9         3          6  35.0  14.507315   6.40000  115.0  0.0  140.0   
     10        3          6  35.0  13.965651   7.14000  115.0  0.0  151.0   
     11        3          6  35.0  15.947102   4.46180  115.0  0.0  134.8   
     12        3          6  35.0  13.840881   4.69100  115.0  0.0  142.0   
     13        3          6  35.0   2.249649  18.95000  115.0  0.0  245.0   
     14        3          6  35.0   6.857738  14.00000  115.0  0.0  215.0   
     15        3          6  35.0  19.004806   2.00000  115.0  0.0   85.0   
     16        3          6  35.0  19.492161   0.90000  115.0  0.0   60.0   
     17        3          6  35.0  19.242015   1.70000  115.0  0.0   75.0   
     18        3          6  35.0  19.184618   1.81000  115.0  0.0   77.0   
     19        3          6  35.0  19.358979   1.65000  115.0  0.0   75.0   
     20        3          6  35.0   9.501888  10.80000  115.0  0.0  195.0   
     21        3          6  35.0  10.900591   8.65000  115.0  0.0  185.0   
     22        3          6  35.0  19.732647   0.65500  115.0  0.0   56.0   
     23        3          6  35.0  13.439753   4.59100  115.0  0.0  152.0   
     24        3          6  35.0  13.636244   4.77050  115.0  0.0  156.1   
     25        3          6  35.0   5.450615  12.99100  115.0  0.0  227.0   
     26        3          6  35.0  17.776815   1.07150  115.0  0.0   66.1   
     27        3          6  35.0  17.944727   0.86000  115.0  0.0   61.0   
     28        3          6  35.0  17.776815   1.07150  115.0  0.0   66.1   
     29        3          6  35.0  17.742291   1.06500  115.0  0.0   66.0   
     30        3          6  35.0  17.776815   1.07150  115.0  0.0   66.1   
...          ...        ...   ...        ...       ...    ...  ...    ...   
93   11        6          6  35.0  18.353990   1.58000  115.0  0.0   81.0   
     12        6          6  35.0  16.348287   4.77500  115.0  0.0  120.0   
     13        6          6  35.0  18.106282   3.00000  115.0  0.0   95.0   
     14        6          6  35.0  18.352494   2.67500  115.0  0.0   90.0   
     15        6          6  35.0  17.829075   3.32500  115.0  0.0  100.0   
     16        6          6  35.0  17.216159   3.97500  115.0  0.0  110.0   
     17        6          6  35.0  16.786546   4.37500  115.0  0.0  115.0   
     18        6          6  35.0  17.510346   3.66000  115.0  0.0  107.0   
     19        6          6  35.0  18.812764   2.17500  115.0  0.0   90.0   
     20        6          6  35.0  17.848249   3.20000  115.0  0.0  105.0   
94   1         9          6  35.0  18.719680   2.20000  115.0  0.0   82.0   
     2         9          6  35.0  14.307001   5.81500  115.0  0.0  146.0   
     3         9          6  35.0  15.672237   5.24000  115.0  0.0  127.0   
     4         9          6  35.0  17.203957   3.90600  115.0  0.0  108.3   
     5         9          6  35.0  14.287087   6.17750  115.0  0.0  143.3   
     6         9          6  35.0  18.472328   2.67500  115.0  0.0   90.0   
     7         9          6  35.0  18.515589   2.66100  115.0  0.0   90.0   
     8         9          6  35.0  18.861407   2.32600  115.0  0.0   85.0   
     9         9          6  35.0  14.976465   5.67600  115.0  0.0  135.0   
     10

### Функции расчёта выигрышей в игре

In [8]:
class Game:
    import numpy as np
    def __init__(self,R,s0,types):
        self.R = R
        self.s0 = np.array(s0)
        self.n = len(s0)
        self.types = np.array(types)
    def u(self,x):
        return self.np.sqrt(self.types+x)
game = Game( 115,(115/3,115/3,115/3), (1,9,25))

class Mechanism:
    def __init__(self,game,params,xfunc,tfunc):
        self.game = game
        for k,v in params.items():
            setattr(self,k,v)
        self.xfunc = xfunc
        self.tfunc = tfunc
        
    def x(self,s):
        return self.xfunc(s,self.game)
    def t(self,s):
        return self.tfunc(s,self.game,self)
    def f(self,s):
        return game.u(self.x(s))-self.t(s)

YHMechanism = Mechanism(game,
                        {'beta':0.0005}, 
                        lambda s,g: s*g.R/s.sum(),
                        lambda s,g,m: m.beta*s*(np.repeat(s.sum(),3)-s))

class GLClass:
    from scipy.spatial import distance
    def __init__(self):
        self.glx = lambda s,g: s.sum(axis=0) / g.n
    def glt(self,s,g,m):
        p = m.beta * self.distance.cdist([self.glx(s,g)],s,'sqeuclidean')
        pmean = m.alfa * p.mean()
        return p - pmean
GL = GLClass()
    
GLMechanism = Mechanism(game,{'beta':0.0005,'alfa':1},
                        GL.glx,
                        GL.glt
                       )

In [10]:
print(YHMechanism.beta)
s = np.array([1.2,40,80])
print(YHMechanism.t(s))
YHMechanism.f(s)

0.0005
[0.072 1.624 1.648]


array([1.39040003, 5.22828395, 8.39727704])

In [31]:
np.positive([-1.,1])

array([-1.,  1.])

In [32]:
def Unash(f,f0):
    df = f-f0
    return (np.fabs(df)).prod() * np.min(np.sign(df))

print(YHMechanism.game.s0)

f0 = YHMechanism.game.u([0,0,0])
print(f0)

Unash(f0,s)

[38.33333333 38.33333333 38.33333333]
[1. 3. 5.]


-39394.799999999996

### Цикл получения исходных данных для Нэш-торгов

In [33]:
import itertools
res = []
for name,group in YHg:
    group = group.sort_values(by='GrSubject')
    if name[1] == 1 :
        curname = name[0]
        prevg = group
        continue
    
    fprev = prevg['Gain'].values
    sprev = prevg['s2'].values
    s = group['s2'].values
    s1, s2, s3 = sprev.copy(), sprev.copy(), sprev.copy()
    s1[0] = s[0]
    s2[1] = s[1]
    s3[2] = s[2]
    f1 = YHMechanism.f(s1)
    f2 = YHMechanism.f(s2)
    f3 = YHMechanism.f(s3)
    U = Unash(fprev,f0)
    U1 = Unash(f1,f0)
    U2 = Unash(f2,f0)
    U3 = Unash(f3,f0)
    Unew = Unash(YHMechanism.f(s),f0)
    res.append( [a for a in itertools.chain(name,[U1>U,U2>U,U3>U,Unew>U,U,U1,U2,U3,Unew])] )
    prevg = group
    #print(name,prev['Gain'])

In [34]:
a = np.vstack(res)
data_a = pd.DataFrame(a); data_a

,0,1,2,3,4,5,6,7,8,9,10
0,40.0,2.0,0.0,1.0,1.0,0.0,-0.166445,-0.182068,-0.166445,-0.166445,-0.182068
1,40.0,3.0,1.0,0.0,0.0,1.0,-0.182068,-0.166445,-0.182068,-0.182068,-0.166445
2,40.0,4.0,0.0,1.0,1.0,1.0,-0.166445,-0.182068,0.663205,0.141136,4.637448
3,40.0,5.0,0.0,0.0,1.0,0.0,4.637448,-2.334728,4.523720,4.758127,-2.365943
4,40.0,6.0,1.0,0.0,1.0,1.0,-2.365943,4.312664,-42.795611,-2.119715,0.061842
5,40.0,7.0,1.0,1.0,0.0,1.0,0.061842,0.064509,2.568412,0.061842,2.765782
6,40.0,8.0,1.0,1.0,1.0,1.0,2.765782,2.955366,2.765782,2.765782,2.955366
7,40.0,9.0,0.0,0.0,1.0,1.0,2.955366,1.349725,2.955366,5.147459,5.049548
8,40.0,10.0,0.0,1.0,1.0,0.0,5.049548,-0.238005,6.277598,11.369736,0.827750
9,40.0,11.0,1.0,0.0,1.0,1.0,0.827750,22.653564,-0.456697,3.488995,3.342179


In [35]:
pda = data_a[[2,3,4,5]]
pdaPar = data_a[data_a[6]>0][[2,3,4,5]]

## YH

Игры с механизмом YH и механизмом остановки "по договорённости" - никто не поменял заявок.
Всего шагов 1035, если брать для каждого игрока в отдельности, то 3105 = 1035 * 3

Функция Нэша: $ U(f,f^0)=\prod_{i} (f_i -f_i^0)') $

Базовые выигрыши: $ f^0 = u(0,0,0) $

Определения для текущего шага $t>1$: $$ U_{prev} = U(f(s^{t-1}),f^0)') $$
$$ U_i =U(f(s_i^t,s_{-i}^{t-1}),f^0) $$
$$ U_{new}=U(f(s^t),f^0) $$

In [36]:
def Nash_stat(data):
    a = (data[data>0].count()).values
    return [a[0],a[1],a[2],data[(data[2]*data[3]*data[4]==1)].shape[0],
                                       data[data[2]*data[3]*data[4]*data[5]==1].shape[0],
                                       data[data[5]==1].shape[0],data[(data[5]==1) & (data[2]*data[3]*data[4]==0)].shape[0],
                                       data[(data[5]==1) & (data[2]+data[3]+data[4]==0)].shape[0]
                                       ]

In [37]:
pd.DataFrame({'Критерии':['U1 > Uprev','U2 > Uprev','U3 > Uprev','U1>Uprev & U2>Uprev & U3>Uprev',
                          'U1>Uprev & U2>Uprev & U3>Uprev & Unew > Uprev','Unew > Uprev',
                          'Unew > Uprev & Exist Ui < Uprev','Unew > Uprev & All Ui < Uprev'],
              'Число подходящих ходов':Nash_stat(pda),
              'Число подх. ходов Uprev>0':Nash_stat(pdaPar)
             })

,Критерии,Число подходящих ходов,Число подх. ходов Uprev>0
0,U1 > Uprev,172,148
1,U2 > Uprev,181,155
2,U3 > Uprev,189,162
3,U1>Uprev & U2>Uprev & U3>Uprev,64,58
4,U1>Uprev & U2>Uprev & U3>Uprev & Unew > Uprev,64,58
5,Unew > Uprev,167,127
6,Unew > Uprev & Exist Ui < Uprev,103,69
7,Unew > Uprev & All Ui < Uprev,1,0


In [13]:
YH40 = YHData[YHData['Game']==40].groupby(['Game','Time'])

In [15]:
YH40.sum()

Group  GrSubject  Type       Gain   Penalty      x   s1     s2  \
Game Time                                                                   
40   1         3          6  35.0   6.812121  11.59900  115.0  0.0  215.0   
     2         3          6  35.0   6.808913  11.62040  115.0  0.0  215.1   
     3         3          6  35.0   6.812121  11.59900  115.0  0.0  215.0   
     4         3          6  35.0  15.951731   2.57165  115.0  0.0  102.6   
     5         3          6  35.0   6.833104  14.15300  115.0  0.0  216.5   
     6         3          6  35.0  11.057325   7.45100  115.0  0.0  172.0   
     7         3          6  35.0  15.000859   3.33200  115.0  0.0  121.1   
     8         3          6  35.0  15.015961   3.34400  115.0  0.0  121.2   
     9         3          6  35.0  14.507315   6.40000  115.0  0.0  140.0   
     10        3          6  35.0  13.965651   7.14000  115.0  0.0  151.0   
     11        3          6  35.0  15.947102   4.46180  115.0  0.0  134.8   
     12        3          6  35.0  13.840881   4.69100  115.0  0.0  142.0   
     13        3          6  35.0   2.249649  18.95000  115.0  0.0  245.0   
     14        3          6  35.0   6.857738  14.00000  115.0  0.0  215.0   
     15        3          6  35.0  19.004806   2.00000  115.0  0.0   85.0   
     16        3          6  35.0  19.492161   0.90000  115.0  0.0   60.0   
     17        3          6  35.0  19.242015   1.70000  115.0  0.0   75.0   
     18        3          6  35.0  19.184618   1.81000  115.0  0.0   77.0   
     19        3          6  35.0  19.358979   1.65000  115.0  0.0   75.0   
     20        3          6  35.0   9.501888  10.80000  115.0  0.0  195.0   
     21        3          6  35.0  10.900591   8.65000  115.0  0.0  185.0   
     22        3          6  35.0  19.732647   0.65500  115.0  0.0   56.0   
     23        3          6  35.0  13.439753   4.59100  115.0  0.0  152.0   
     24        3          6  35.0  13.636244   4.77050  115.0  0.0  156.1   
     25        3          6  35.0   5.450615  12.99100  115.0  0.0  227.0   
     26        3          6  35.0  17.776815   1.07150  115.0  0.0   66.1   
     27        3          6  35.0  17.944727   0.86000  115.0  0.0   61.0   
     28        3          6  35.0  17.776815   1.07150  115.0  0.0   66.1   
     29        3          6  35.0  17.742291   1.06500  115.0  0.0   66.0   
     30        3          6  35.0  17.776815   1.07150  115.0  0.0   66.1   
     31        3          6  35.0  17.742291   1.06500  115.0  0.0   66.0   
     32        3          6  35.0  17.598476   3.60000  115.0  0.0  105.0   
     33        3          6  35.0  17.116473   4.05500  115.0  0.0  112.0   
     34        3          6  35.0  17.042679   4.11100  115.0  0.0  113.0   
     35        3          6  35.0  16.707016   4.48500  115.0  0.0  118.0   
     36        3          6  35.0  18.194570   2.78400  115.0  0.0  101.0   
     37        3          6  35.0   7.160956  13.34400  115.0  0.0  211.0   
     38        3          6  35.0  17.310031   1.77500  115.0  0.0  130.0   
     39        3          6  35.0  19.647432   0.69500  115.0  0.0   56.0   
     40        3          6  35.0  19.609865   0.65000  115.0  0.0   55.0   
     41        3          6  35.0  19.647432   0.69500  115.0  0.0   56.0   
     42        3          6  35.0  18.736173   1.65000  115.0  0.0   75.0   
     43        3          6  35.0  16.748558   3.15500  115.0  0.0  116.0   
     44        3          6  35.0  16.771674   3.05000  115.0  0.0  115.0   
     45        3          6  35.0  18.235891   1.33500  115.0  0.0   96.0   
     46        3          6  35.0   8.222200  11.15000  115.0  0.0  205.0   
     47        3          6  35.0  17.645870   1.39100  115.0  0.0  127.0   
     48        3          6  35.0  18.736173   1.65000  115.0  0.0   75.0   
     49        3          6  35.0  18.439013   1.97000  115.0  0.0   81.0   
     50        3          6  35.0  17.845657   2.70000  115.0  0.0   95.0   
     51